<a href="https://colab.research.google.com/github/felixbmuller/nlp-commonsense/blob/main/NLP_Commonsense_Assignment_2_Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2 - 1

## Setup

In [3]:
!pip install -q transformers datasets torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.2 MB/s eta 0:00:00


In [4]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
!apt install git-lfs >/dev/null

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [6]:
import transformers

print(transformers.__version__)

4.26.1


Set configuration

In [7]:
model_checkpoint = "bert-base-uncased"
batch_size = 16

## Load and investigate the dataset

In [8]:
from datasets import load_dataset, load_metric
import pandas as pd

In [9]:
datasets = load_dataset("super_glue", "copa")

datasets

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset super_glue downloaded and prepared to /root/.cache/huggingface/datasets/super_glue/copa/1.0.3/bb9675f958ebfee0d5d6dc5476fafe38c79123727a7258d515c450873dbdbbed. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'choice1', 'choice2', 'question', 'idx', 'label'],
        num_rows: 400
    })
    validation: Dataset({
        features: ['premise', 'choice1', 'choice2', 'question', 'idx', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['premise', 'choice1', 'choice2', 'question', 'idx', 'label'],
        num_rows: 500
    })
})

In [10]:
df = pd.DataFrame(datasets["train"])
df.head(10)

,premise,choice1,choice2,question,idx,label
0,My body cast a shadow over the grass.,The sun was rising.,The grass was cut.,cause,0,0
1,The woman tolerated her friend's difficult beh...,The woman knew her friend was going through a ...,The woman felt that her friend took advantage ...,cause,1,0
2,The women met for coffee.,The cafe reopened in a new location.,They wanted to catch up with each other.,cause,2,1
3,The runner wore shorts.,The forecast predicted high temperatures.,She planned to run along the beach.,cause,3,0
4,The guests of the party hid behind the couch.,It was a surprise party.,It was a birthday party.,cause,4,0
5,The politician lost the election.,He ran negative campaign ads.,No one voted for him.,cause,5,1
6,The stain came out of the shirt.,I patched the shirt.,I bleached the shirt.,cause,6,1
7,The man got a discount on his groceries.,He greeted the cashier.,He used a coupon.,cause,7,1
8,The physician misdiagnosed the patient.,The patient filed a malpractice lawsuit agains...,The patient disclosed confidential information...,effect,8,0
9,The customer filed a complaint with the store ...,The sales associate undercharged the customer.,The sales associate acted rude to the customer.,cause,9,1


Make sure that there are only two types of questions (cause and effect), as we will hardcode their treatment in the next section.

In [11]:
df.question.unique()

array(['cause', 'effect'], dtype=object)

## Preprocessing the data

In [12]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [13]:
ending_names = ["choice1", "choice2"]

# Change cause and effect to a proper question
QUESTION_MAP = {
    "cause": "What was the cause of this?",
    "effect": "What happened as a RESULT?",
}

def preprocess_function(examples):
    # Repeat premise and question twice for both possible answers
    first_sentences = [[f"{context} {QUESTION_MAP[question]}"] * 2 for context, question in zip(examples["premise"], examples["question"])]
    # Grab all second sentences possible for each context.
    second_sentences = [[c1, c2] for c1, c2 in zip(examples["choice1"], examples["choice2"])]
    
    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    if not len(first_sentences) == len(second_sentences):
        raise ValueError("lengths dont match")
    
    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    return {k: [v[i:i+2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists of lists for each key: a list of all examples (here 5), then a list of all choices (4) and a list of input IDs (length varying here since we did not apply any padding):

### Test Tokenizer and Preprocessing

In [14]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
examples = datasets["train"][:5]
features = preprocess_function(examples)

print(features.keys())
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
5 2 [24, 24]


To check we didn't do anything group when grouping all possibilites then unflattening, let's have a look at the decoded inputs for a given example:

In [16]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(2)]

['[CLS] the runner wore shorts. what was the cause of this? [SEP] the forecast predicted high temperatures. [SEP]',
 '[CLS] the runner wore shorts. what was the cause of this? [SEP] she planned to run along the beach. [SEP]']

We can compare it to the ground truth:

In [17]:
datasets["train"][3]

{'premise': 'The runner wore shorts.',
 'choice1': 'The forecast predicted high temperatures.',
 'choice2': 'She planned to run along the beach.',
 'question': 'cause',
 'idx': 3,
 'label': 0}

### Apply Preprocessing to the Whole Dataset

In [18]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

## Fine-tuning the model

In [19]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch
import numpy as np

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-copa",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch


def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

When called on a list of examples, it will flatten all the inputs/attentions masks etc. in big lists that it will pass to the `tokenizer.pad` method. This will return a dictionary with big tensors (of shape `(batch_size * 4) x seq_length`) that we then unflatten.

### Test Collator

We can check this data collator works on a list of features, we just have to make sure to remove all features that are not inputs accepted by our model (something the `Trainer` will do automatically for us after):

In [20]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Again, all those flatten/un-flatten are sources of potential errors so let's make another sanity check on our inputs:

In [21]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(2)]

['[CLS] the physician misdiagnosed the patient. what happened as a result? [SEP] the patient filed a malpractice lawsuit against the physician. [SEP] [PAD]',
 '[CLS] the physician misdiagnosed the patient. what happened as a result? [SEP] the patient disclosed confidential information to the physician. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]']

In [22]:
datasets["train"][8]

{'premise': 'The physician misdiagnosed the patient.',
 'choice1': 'The patient filed a malpractice lawsuit against the physician.',
 'choice2': 'The patient disclosed confidential information to the physician.',
 'question': 'effect',
 'idx': 8,
 'label': 0}

### Run Training

In [23]:
trainer.train()

#model.push_to_hub("felixbmuller/bert-base-uncased-finetuned-copa")

The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: premise, choice1, question, idx, choice2. If premise, choice1, question, idx, choice2 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 400
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 75
  Number of trainable parameters = 109483009


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.678627,0.650000
2,No log,0.604108,0.720000
3,No log,0.593839,0.700000


The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: premise, choice1, question, idx, choice2. If premise, choice1, question, idx, choice2 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: premise, choice1, question, idx, choice2. If premise, choice1, question, idx, choice2 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: premise, choice1, question, idx, choice2. If premise, choice1, question, idx, choice2 are not 

TrainOutput(global_step=75, training_loss=0.569761962890625, metrics={'train_runtime': 18.8639, 'train_samples_per_second': 63.613, 'train_steps_per_second': 3.976, 'total_flos': 36407665392768.0, 'train_loss': 0.569761962890625, 'epoch': 3.0})

## Evalute the Model


In [24]:
predictions, label_ids, metrics = trainer.predict(encoded_datasets["validation"], metric_key_prefix="val")

The following columns in the test set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: premise, choice1, question, idx, choice2. If premise, choice1, question, idx, choice2 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 100
  Batch size = 16


In [25]:
metrics

{'val_loss': 0.5938388109207153,
 'val_accuracy': 0.699999988079071,
 'val_runtime': 0.4463,
 'val_samples_per_second': 224.062,
 'val_steps_per_second': 15.684}

In [26]:
val = pd.DataFrame(datasets["validation"])
val["label_ids"] = label_ids
val["pred0"] = predictions[:, 0]
val["pred1"] = predictions[:, 1]
val["pred_label"] = np.argmax(predictions, axis=1)

Sanity check to ensure that predictions work the way I expect them to do

In [27]:
print("accuracy", (val.pred_label == val.label).astype(np.float32).mean())

accuracy 0.7


In [28]:
# from google.colab import drive
import joblib
# drive.mount('/content/drive')

In [30]:
joblib.dump(val, "./bert-case-uncased-finetuned-copa-baseline-validation-results.joblib")

['./bert-case-uncased-finetuned-copa-baseline-validation-results.joblib']

In [32]:
val.head(24)

,premise,choice1,choice2,question,idx,label,label_ids,pred0,pred1,pred_label
0,The man turned on the faucet.,The toilet filled with water.,Water flowed from the spout.,effect,0,1,1,0.380501,0.330910,0
1,The girl found a bug in her cereal.,She poured milk in the bowl.,She lost her appetite.,effect,1,1,1,-2.204281,0.220649,1
2,The woman retired.,She received her pension.,She paid off her mortgage.,effect,2,0,0,0.646664,0.538157,0
3,I wanted to conserve energy.,I swept the floor in the unoccupied room.,I shut off the light in the unoccupied room.,effect,3,1,1,-0.955725,-1.183087,0
4,The hamburger meat browned.,The cook froze it.,The cook grilled it.,cause,4,1,1,0.253277,0.513073,1
5,I doubted the salesman's pitch.,I turned his offer down.,He persuaded me to buy the product.,effect,5,0,0,-0.583832,0.375512,1
6,I decided to stay home for the night.,The forecast called for storms.,My friends urged me to go out.,cause,6,0,0,-0.375828,-0.249945,1
7,My eyes became red and puffy.,I was sobbing.,I was laughing.,cause,7,0,0,0.293778,-1.646373,0
8,The flame on the candle went out.,I blew on the wick.,I put a match to the wick.,cause,8,0,0,-0.276586,0.338281,1
9,The man drank heavily at the party.,He had a headache the next day.,He had a runny nose the next day.,effect,9,0,0,-1.396467,-1.716648,0


In [33]:
wrong_samples = val[val.label !=  val.pred_label]
wrong_samples.sample(25, random_state=42)

,premise,choice1,choice2,question,idx,label,label_ids,pred0,pred1,pred_label
96,I tidied up my house.,I was swamped with work.,I was expecting company.,cause,96,1,1,-1.044998,-1.708671,0
49,I pressed my hand into the wet cement.,My handprint dried in the cement.,Cracks emerged in the cement.,effect,49,0,0,-0.041308,0.267752,1
70,The woman asked the man to leave.,He insulted her.,He thanked her.,cause,70,0,0,-1.496410,0.400892,1
58,The man fainted.,He took a nap.,He ran a marathon.,cause,58,1,1,-1.386704,-2.000019,0
25,The driver got a flat tire.,He went over the speed limit.,He ran over a nail.,cause,25,1,1,0.548932,-1.613890,0
26,My view of the movie screen was blocked.,The couple behind me was whispering.,A tall person was sitting in front of me.,cause,26,1,1,-0.451442,-0.728329,0
97,The airline mishandled my luggage.,They offered me compensation.,They cancelled my flight.,effect,97,0,0,0.601630,0.675739,1
72,The teacher assigned homework to the students.,The students passed notes.,The students groaned.,effect,72,1,1,-1.246671,-1.853473,0
36,The couple travelled south for the winter.,They were retired.,They were separated.,cause,36,0,0,-2.045467,-1.760252,1
0,The man turned on the faucet.,The toilet filled with water.,Water flowed from the spout.,effect,0,1,1,0.380501,0.330910,0
